## Nama : Muhammad Bastian Hanafi
## NIM : 20.11.3714
## Kelas : 20 IF 07

## install pyspark

In [ ]:
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=e4dafa1828852cfbae117389fdaa319222fd89140e822a66097ccd4d3737ec54
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_iris
import pandas as pd

In [ ]:
iris = load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['label'] = pd.Series(iris.target)
print(df_iris.head())


   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   label  
0      0  
1      0  
2      0  
3      0  
4      0  


In [ ]:
sc = SparkContext().getOrCreate()
sqlContext = SQLContext(sc)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
data = sqlContext.createDataFrame(df_iris)
print(data.printSchema())

root
 |-- sepal length (cm): double (nullable = true)
 |-- sepal width (cm): double (nullable = true)
 |-- petal length (cm): double (nullable = true)
 |-- petal width (cm): double (nullable = true)
 |-- label: long (nullable = true)

None


In [ ]:
features = iris.feature_names

In [20]:
features

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [ ]:
va = VectorAssembler(inputCols = features, outputCol='features')

In [ ]:
va_df = va.transform(data)
va_df = va_df.select(['features', 'label'])
va_df.show(3)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|    0|
|[4.9,3.0,1.4,0.2]|    0|
|[4.7,3.2,1.3,0.2]|    0|
+-----------------+-----+
only showing top 3 rows



In [ ]:
(train, test) = va_df.randomSplit([0.9, 0.1])

In [ ]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
nb = nb.fit(train)

In [ ]:
pred = nb.transform(test)
pred.show(3)

+-----------------+-----+--------------------+--------------------+----------+
|         features|label|       rawPrediction|         probability|prediction|
+-----------------+-----+--------------------+--------------------+----------+
|[4.9,2.4,3.3,1.0]|    1|[-17.137344274704...|[0.11771136896092...|       1.0|
|[5.0,3.0,1.6,0.2]|    0|[-11.666370042342...|[0.68174017940859...|       0.0|
|[5.5,3.5,1.3,0.2]|    0|[-11.985567631537...|[0.80003241170663...|       0.0|
+-----------------+-----+--------------------+--------------------+----------+
only showing top 3 rows



In [ ]:
evaluator=MulticlassClassificationEvaluator(predictionCol="prediction")
acc = evaluator.evaluate(pred)
print("Prediction Accuracy: ", acc)

Prediction Accuracy:  0.934141414141414


In [ ]:
y_pred=pred.select("prediction").collect()
y_orig=pred.select("label").collect()

y_pred

[Row(prediction=1.0),
 Row(prediction=0.0),
 Row(prediction=0.0),
 Row(prediction=1.0),
 Row(prediction=1.0),
 Row(prediction=1.0),
 Row(prediction=1.0),
 Row(prediction=1.0),
 Row(prediction=2.0),
 Row(prediction=2.0),
 Row(prediction=2.0),
 Row(prediction=2.0),
 Row(prediction=1.0),
 Row(prediction=2.0),
 Row(prediction=2.0)]

In [ ]:
cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[2 0 0]
 [0 7 1]
 [0 0 5]]
